![trainers in a store](trainers.jpg)

Sports clothing and athleisure attire is a huge industry, worth approximately [$193 billion in 2021](https://www.statista.com/statistics/254489/total-revenue-of-the-global-sports-apparel-market/) with a strong growth forecast over the next decade! 

In this notebook, we will undertake the role of a product analyst for an online sports clothing company. The company is specifically interested in how it can improve revenue. We will dive into product data such as pricing, reviews, descriptions, and ratings, as well as revenue and website traffic, to produce recommendations for its marketing and sales teams.  

We will use the datasets given below.

#  brands.csv

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `brand` | Brand of the product | 

# finance.csv

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `listing_price` | Original price of the product | 
| `sale_price` | Discounted price of the product |
| `discount` | Discount off the listing price, as a decimal | 
| `revenue` | Revenue generated by the product |

# info.csv

| Columns | Description |
|---------|-------------|
| `product_name` | Name of the product | 
| `product_id` | Unique product identifier |
| `description` | Description of the product |

# reviews.csv

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `rating` | Average product rating | 
| `reviews` | Number of reviews for the product |

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load the datasets from CSV files
brands = pd.read_csv("brands.csv") 
finance = pd.read_csv("finance.csv")
info = pd.read_csv("info.csv")
reviews = pd.read_csv("reviews.csv")

# Merging the datasets on the common column "product_id"
# First, merge info with finance, then reviews, and finally brands
merged_df = info.merge(finance, on="product_id")
merged_df = merged_df.merge(reviews, on="product_id")
merged_df = merged_df.merge(brands, on="product_id")

# Removing rows with missing values (NaN) to clean the data
merged_df.dropna(inplace=True)

# Create a new column 'price_label' which bins 'listing_price' into 4 quartiles
# Labels that I will use are: "Budget", "Average", "Expensive", and "Elite"
merged_df['price_label'] = pd.qcut(merged_df['listing_price'], q=4, labels=["Budget","Average","Expensive","Elite"])

# Group the data by 'brand' and 'price_label' to get the number of products and average revenue
# Aggregate the number of products and mean revenue, rounding to 2 decimal places
adidas_vs_nike = merged_df.groupby(['brand','price_label'], as_index=False).agg(num_products=('price_label','count'),mean_revenue=('revenue', 'mean')).round(2)

# Create a new column 'desc_length' which calculates the length of the product description
merged_df['desc_length'] = merged_df['description'].str.len()

# Define the bins and labels for categorizing the description lengths into intervals of 100 characters
bins = [0, 100, 200, 300, 400, 500, 600, 700]
labels = ['100', '200', '300', '400', '500', '600', '700']

# Bin the description lengths into categories and create a new column 'description_length'
merged_df['description_length'] = pd.cut(merged_df['desc_length'], bins=bins, labels=labels, include_lowest=True)

# Group the data by 'description_length' to calculate mean rating and count the number of reviews
# Aggregate the mean rating and number of reviews, rounding to 2 decimal places
description_lengths = merged_df.groupby('description_length', as_index=False).agg(mean_rating=('rating', 'mean'),num_reviews=('reviews', 'count')).round(2)

#Viewing our results.
print(description_lengths)
print(adidas_vs_nike)


  description_length  mean_rating  num_reviews
0                100         2.26            7
1                200         3.19          526
2                300         3.28         1785
3                400         3.29          651
4                500         3.35          118
5                600         3.12           15
6                700         3.65           15
    brand price_label  num_products  mean_revenue
0  Adidas      Budget           574       2015.68
1  Adidas     Average           655       3035.30
2  Adidas   Expensive           759       4621.56
3  Adidas       Elite           587       8302.78
4    Nike      Budget           357       1596.33
5    Nike     Average             8        675.59
6    Nike   Expensive            47        500.56
7    Nike       Elite           130       1367.45
